In [1]:
import pandas as pd

# Подключаю библиотеку

In [15]:
bookings = pd.read_csv('https://stepik.org/media/attachments/lesson/360344/bookings.csv', 
                    encoding = 'windows-1251', 
                    sep = ';')

In [21]:
bookings_head =  bookings.head(7)

In [23]:
bookings.shape

(119390, 21)

In [25]:
bookings.dtypes.value_counts()

object     10
int64      10
float64     1
dtype: int64

# Заменяю строки с помощью функции

In [29]:
def to_lower_underscore(name):
    name = name.lower().replace(' ','_')
    return name

In [31]:
bookings = bookings.rename(columns = to_lower_underscore )

In [33]:
bookings.head()

,hotel,is_canceled,lead_time,arrival_full_date,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,...,adults,children,babies,meal,country,reserved_room_type,assigned_room_type,customer_type,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015-07-01,2015,July,27,1,0,0,...,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015-07-01,2015,July,27,1,0,0,...,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015-07-01,2015,July,27,1,0,1,...,1,0.0,0,BB,GBR,A,C,Transient,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015-07-01,2015,July,27,1,0,1,...,1,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015-07-01,2015,July,27,1,0,2,...,2,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-03


## овербукинг



In [48]:
bookings.query('assigned_room_type != reserved_room_type').shape[0]

14917

# На какой месяц чаще всего оформляли бронь в 2016 году?

In [54]:
bookings.query('arrival_date_year == 2016')\
        .arrival_date_month\
        .value_counts()
        

October      6203
May          5478
April        5428
September    5394
June         5292
August       5063
March        4824
July         4572
November     4454
February     3891
December     3860
January      2248
Name: arrival_date_month, dtype: int64

# На какой месяц чаще всего оформляли бронь в 2017 году?

In [53]:
bookings.query('arrival_date_year == 2017')\
        .arrival_date_month\
        .value_counts()

May         6313
April       5661
June        5647
July        5313
March       4970
August      4925
February    4177
January     3681
Name: arrival_date_month, dtype: int64

# На какой месяц бронирования отеля типа City Hotel отменялись чаще всего в 2015? 2016? 2017? 

In [61]:
(
    
    bookings.query('(hotel == "City Hotel") and (is_canceled == 1)')
            .groupby('arrival_date_year')
            .arrival_date_month	
            .value_counts()
)

arrival_date_year  arrival_date_month
2015               September             1543
                   October               1321
                   August                1232
                   July                   939
                   December               668
                   November               301
2016               October               1947
                   June                  1720
                   September             1567
                   April                 1539
                   May                   1436
                   November              1360
                   August                1247
                   March                 1108
                   December              1072
                   July                  1043
                   February               930
                   January                438
2017               May                   2217
                   April                 1926
                   June                  1

# Посмотрите на числовые характеристики трёх колонок: adults, children и babies. Какая из них имеет наибольшее среднее значение?

In [70]:
bookings[['adults', 'children', 'babies']].describe()


,adults,children,babies
count,119390.000000,119386.000000,119390.000000
mean,1.856403,0.103890,0.007949
std,0.579261,0.398561,0.097436
min,0.000000,0.000000,0.000000
25%,2.000000,0.000000,0.000000
50%,2.000000,0.000000,0.000000
75%,2.000000,0.000000,0.000000
max,55.000000,10.000000,10.000000


# Создайте колонку total_kids, объединив столбцы children и babies. 

In [71]:
bookings['total_kids'] = bookings['children'] + bookings['babies']

In [ ]:
(
bookings.groupby('hotel')
        .agg({'total_kids': 'mean'})
        .round(decimals=2)
        .max()
)
    

#### Churn rate (отток, коэффициент оттока) – это процент подписчиков (например, на push-уведомления от сайта), которые отписались от канала коммуникации, отказались от услуг сервиса в течение определенного периода времени. Иными словами, представляет собой отношение количества ушедших пользователей к общему количеству пользователей, выраженное в процентах.

In [ ]:
bookings['has_kids'] = bookings.total_kids > 0

In [104]:

no_kids_churn = round(bookings.query('is_canceled == 1 and has_kids == False').shape[0]\
    / bookings.query('has_kids == False').shape[0] * 100, 2)



In [105]:

yes_kids_churn = round(bookings.query('is_canceled == 1 and has_kids == True').shape[0]\
    / bookings.query('has_kids == False').shape[0] * 100, 2)



In [ ]:
no_kids_churn, yes_kids_churn
